### Conversion to ONNX
You need to install transformers to this particular commit,

In [ ]:
%pip install --upgrade git+https://github.com/huggingface/transformers.git@2e35bac4e73558d334ea5bbf96a1116f7c0d7fb3
# # !pip install transformers
# !pip install transformers[onnx]
# # pip install -U tensorflow==2.10 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git (to revision 2e35bac4e73558d334ea5bbf96a1116f7c0d7fb3) to /tmp/pip-req-build-lc6pu_oh
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-lc6pu_oh
  Running command git rev-parse -q --verify 'sha^2e35bac4e73558d334ea5bbf96a1116f7c0d7fb3'
  Running command git fetch -q https://github.com/huggingface/transformers.git 2e35bac4e73558d334ea5bbf96a1116f7c0d7fb3
  Running command git checkout -q 2e35bac4e73558d334ea5bbf96a1116f7c0d7fb3
  Resolved https://github.com/huggingface/transformers.git to commit 2e35bac4e73558d334ea5bbf96a1116f7c0d7fb3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━

Check if the code has been added to an stable version, it seems that commit has been merged already,

https://github.com/huggingface/transformers/pull/19254

I tried different tolerances, I remember we didn't need to go as large as 1e-2, but something in between 1e-2 and 1e-3, I tried all these(don't remember exactly which one was the successful one)

In [ ]:
# Working examples:
!python -m transformers.onnx --model=distilbert-base-uncased onnx/

2023-05-10 06:33:42.504294: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 06:33:43.724903: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
!python -m transformers.onnx --model=naver-clova-ix/donut-proto-finetuned-docvqa --feature=vision2seq-lm scratch/onnx --atol 1e-3

In [ ]:
!python -m transformers.onnx --model=google/pix2struct-docvqa-base --feature=vision2seq-lm scratch/onnx --atol 1e-3

# !python -m transformers.onnx --model=naver-clova-ix/donut-base-finetuned-cord-v2 --feature=vision2seq-lm scratch/onnx --atol 1e-2
# !python -m transformers.onnx --model=naver-clova-ix/donut-base-finetuned-cord-v2 --feature=vision2seq-lm scratch/onnx --atol 1e-3
# !python -m transformers.onnx --model=naver-clova-ix/donut-base-finetuned-cord-v2 --feature=vision2seq-lm scratch/onnx --atol 1e-3

# !python -m transformers.onnx --model=naver-clova-ix/donut-base-finetuned-docvqa --feature=vision2seq-lm docvqa/onnx --atol 3e-3
# !python -m transformers.onnx --model=naver-clova-ix/donut-base-finetuned-docvqa --feature=vision2seq-lm docvqa/onnx --atol 6e-3
# !python3.7 -m transformers.onnx --model=naver-clova-ix/donut-base-finetuned-docvqa --feature=vision2seq-lm docvqa/onnx --atol 7e-3
# !python3.7 -m transformers.onnx --model=naver-clova-ix/donut-base-finetuned-docvqa --feature=vision2seq-lm docvqa/onnx --atol 7e-3

In [ ]:
!python -m pip install optimum

In [ ]:
import os
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
save_directory = "tmp/onnx/"
file_name = "model.onnx"
onnx_path = os.path.join(save_directory, "model.onnx")

# Load a model from transformers and export it through the ONNX format
model = ORTModelForSequenceClassification.from_pretrained(model_checkpoint, from_transformers=True)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Save the onnx model and tokenizer
model.save_pretrained(save_directory, file_name=file_name)
tokenizer.save_pretrained(save_directory)

This one is for taking the model from disk,

In [ ]:
model_ckpt = "path_to_your_checkpoint"
!python3.7 -m transformers.onnx --model={model_ckpt} --feature=vision2seq-lm onnx/ --atol 1e-3

### Quantizing
#### decoder

In [ ]:
!olive optimize --model_path decoder_model.onnx --input_names input_ids,attention_mask,encoder_hidden_states --input_shapes [[-1,-1],[-1,-1],[-1,-1,1024]] --quantization_enabled --dynamic_batching_size 2

#### encoder/decoder with optimum

In [ ]:
from optimum.onnxruntime import AutoQuantizationConfig
from onnxruntime.quantization.qdq_quantizer import QDQQuantizer
from onnxruntime.quantization.onnx_quantizer import ONNXQuantizer
from optimum.onnxruntime import ORTQuantizableOperator
import onnx

quantization_config = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)
quantizer_factory = QDQQuantizer if False else ONNXQuantizer

# here pick either model
path = "./decoder.onnx"
onnx_model = onnx.load(path)
quantizer = quantizer_factory(
                model=onnx_model,
                static=quantization_config.is_static,
                per_channel=quantization_config.per_channel,
                mode=quantization_config.mode,
                weight_qType=quantization_config.weights_dtype,
                activation_qType=quantization_config.activations_dtype,
                tensors_range=None,
                reduce_range=quantization_config.reduce_range,
                nodes_to_quantize=quantization_config.nodes_to_quantize,
                nodes_to_exclude=quantization_config.nodes_to_exclude,
                op_types_to_quantize=[
                    operator.value if isinstance(operator, ORTQuantizableOperator) else operator
                    for operator in quantization_config.operators_to_quantize
                ],
                extra_options={
                    "WeightSymmetric": quantization_config.weights_symmetric,
                    "ActivationSymmetric": quantization_config.activations_symmetric,
                    "EnableSubgraph": False,
                    "ForceSymmetric": quantization_config.activations_symmetric
                    and quantization_config.weights_symmetric,
                    "AddQDQPairToWeight": quantization_config.qdq_add_pair_to_weight,
                    "DedicatedQDQPair": quantization_config.qdq_dedicated_pair,
                    "QDQOpTypePerChannelSupportToAxis": quantization_config.qdq_op_type_per_channel_support_to_axis,
                },
            )

quantizer.quantize_model()
quantizer.model.save_model_to_file("quantized_by_optimum.onnx")

### Handling the memory issues
You should go to this file,

/home/ubuntu/.local/lib/python3.7/site-packages/transformers/onnx/convert.py

in method `validate_model_outputs()`

and patch things similar to the changes here.
This is just reducing the batch size from 3 to 1. I we really want to try 3 different examples, we could just run the validation 3 times, each time with batch size 1, so we don't kill the memory.

In [ ]:
    # generate inputs with a different batch_size and seq_len that was used for conversion to properly test
    # dynamic input shapes.
    if is_torch_available() and issubclass(type(reference_model), PreTrainedModel):
        reference_model_inputs = config.generate_dummy_inputs(
            preprocessor,
            batch_size=1, #config.default_fixed_batch + 1,
            seq_length=config.default_fixed_sequence + 1,
            framework=TensorType.PYTORCH,
        )
    else:
        reference_model_inputs = config.generate_dummy_inputs(
            preprocessor,
            batch_size=1, #config.default_fixed_batch + 1,
            seq_length=config.default_fixed_sequence + 1,
            framework=TensorType.TENSORFLOW,
        )
